In [4]:
import matplotlib.pylab as plt
import numpy as np
import sklearn as sk
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder


#Load the CSV file
data = pd.read_csv('Data_EV.csv')


def Create_data(one_hot):

    #Prepare the data
    X = data.drop(columns=['Number of Evs 2022'])
    y = data['Number of Evs 2022']  # Dependent variable
    X = X.drop(columns=['State'])

    if one_hot == False:
        return X, y
    else:
        encoder = OneHotEncoder(sparse=False)

        one_hot_encoded = encoder.fit_transform(data[['Political Affiliation']])
        feature_names = encoder.categories_[0]
        one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=feature_names)
        X = pd.concat([X, one_hot_encoded_df], axis=1)
        X = X.drop(columns=['Political Affiliation'])

        return X, y


X,y = Create_data(True)
# Standardize the features

#Scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Scale y
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))


c:\Users\fenst\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:


# Assuming X_train, X_test, y_train, y_test are your preprocessed features and target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

# Define the neural network architecture
class EVModel(nn.Module):
    def __init__(self, input_size):
        super(EVModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model
input_size = X_train.shape[1]
model = EVModel(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        inputs = X_train_tensor[i:i+batch_size]
        targets = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    mse = criterion(y_pred_tensor, y_test_tensor)
    mae = torch.mean(torch.abs(y_pred_tensor - y_test_tensor))
    print(f"Mean Squared Error: {mse.item():.4f}")
    print(f"Mean Absolute Error: {mae.item():.4f}")

# Convert predictions and ground truth back to numpy arrays
predictions = y_pred_tensor.numpy()
y_test_numpy = y_test_tensor.numpy()

# Calculate additional metrics if needed
rmse = np.sqrt(mean_squared_error(y_test_numpy, predictions))
print(f"Root Mean Squared Error: {rmse:.4f}")

Epoch [10/50], Loss: 0.0109
Epoch [20/50], Loss: 0.0015
Epoch [30/50], Loss: 0.0011
Epoch [40/50], Loss: 0.0002
Epoch [50/50], Loss: 0.0000
Mean Squared Error: 0.0626
Mean Absolute Error: 0.1386
Root Mean Squared Error: 0.2502
